In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D 
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import glob, pdb, os, sys, cv2
from ripser import ripser
from persim import plot_diagrams
#import gudhi as gd
#from angio import angio_abm, radial_persistence, level_set_flooding, Persist_im
import matplotlib as mpl
from sklearn import svm
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split,cross_val_score


from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf') 

from custom_function import *

In [ ]:
chi_vec = np.round(np.linspace(0,.5,11),2)
hapt_vec = np.round(np.linspace(0,.5,11),2)

num_data_sets = 10*len(chi_vec)*len(hapt_vec)

#length of sprouts (vessel segments) and tips
feature_len = 50

### Initialize lists containing the standard descriptor vectors
sprouts = []
tips = []
vessel_length = []

x = np.linspace(0,1,201)
y = np.linspace(0,1,201)
Y,X = np.meshgrid(x,y)

num_vecs = 1


for i in np.arange(num_vecs):
    sprouts.append(np.zeros((num_data_sets,feature_len)))
    tips.append(np.zeros((num_data_sets,feature_len)))
    vessel_length.append(np.zeros((num_data_sets,1)))

#initialize realization, chi, rho, real labels for each simulation    
labels = np.zeros((num_data_sets,))
chi_real_vec = np.zeros((num_data_sets,))
rho_real_vec = np.zeros((num_data_sets,))
real_vec = np.zeros((num_data_sets,))
    
count = 0
for i,chi in enumerate(chi_vec):
    for j,hapt in enumerate(hapt_vec):
        for real in np.arange(10):
            
            
            
            mat = np.load("results/angio_bio_data_IC_linear_rho_"+str(hapt)+"_chi_"+str(chi)+"_real_"+str(real)+".npy",allow_pickle=True,encoding="latin1").item()
            
            
            sprout = mat['sprouts'][::len(mat['sprouts'])//feature_len]
            sprouts[0][count,:] = sprout[:feature_len]
            
            tip = mat['active_tips'][::len(mat['active_tips'])//feature_len]
            tips[0][count,:] = tip[:feature_len]
            
            vessel_length[0][count,:] = np.max(X[mat['N']==1])
            
            chi_real_vec[count] = chi
            rho_real_vec[count] = hapt
            labels[count] = len(hapt_vec)*i + j
            
            real_vec[count] = real

                
            count+=1

## Running classification scheme

In [ ]:
X_vec = [tips[0],vessel_length[0],sprouts[0],np.hstack((tips[0],vessel_length[0],sprouts[0]))]
title_vec = ["Tips","Vessel Length","Vessel Segments","All"]
title_vec_fig = ["Tips","Vessel length","Vessel segments","All bio summaries"]


#### perform clustering
#filename = "Classes_"+title_vec[i]
for i,X in enumerate(X_vec):
    kmeans_classes,acc,acc_in_sample,centers = clustering_fine_train_test(X,chi_real_vec,rho_real_vec,
                                                real_vec,num_clusters=5,
                                                filename = "param_clustering_"+title_vec[i],title=title_vec_fig[i])

    print(title_vec[i] + ", acc = " + str(acc))



## Creating Accuracy Table

In [ ]:
X_vec = [tips[0],vessel_length[0],sprouts[0],np.hstack((tips[0],vessel_length[0],sprouts[0]))]
title_vec = ["Tips","Vessel Length","Vessel Segments","All"]
title_vec_fig = ["Tips","Vessel length","Vessel segments","All bio summaries"]


create_latex_table_classification_sort(X_vec,title_vec,chi_real_vec,
                                  rho_real_vec,real_vec,num_clusters=5)
